In [1]:
# -*- coding: encoding -*- 

In [169]:
from nltk.corpus import brown
import numpy as np
from collections import defaultdict, OrderedDict
from nltk.corpus import stopwords
import string
from sklearn.cluster import KMeans
from nltk.cluster import cosine_distance, GAAClusterer, KMeansClusterer, euclidean_distance
import sys

In [80]:
texts = map(lambda word: word.encode("utf-8").lower(), brown.words())
# Remove stop words, punctuation and sth doesnt contains any digits and letters like "--"
# Leave the digits
words = filter(lambda word: any(c.isalnum() for c in word) and word not in set(stopwords.words('english')+list(string.punctuation)), texts)

wordCount = defaultdict(int)
for word in words:
    wordCount[word] += 1    
wordCount = OrderedDict(sorted(wordCount.items(), key=lambda t: t[1], reverse = True))

In [82]:
numW = 5000
numC = 1000

In [86]:
w_words = wordCount.keys()[:numW]
c_words = wordCount.keys()[:numC]
w_wordToIdx = {}
c_wordToIdx = {}
for i in xrange(0, numW):
    w_wordToIdx[w_words[i]] = i
for i in xrange(0, numC):
    c_wordToIdx[c_words[i]] = i

In [87]:
pr_c_count = np.zeros(numC)
pr_cw_count = np.zeros((numW,numC))

In [91]:
wordsLength = len(words)

In [94]:
for i in xrange(0, wordsLength):
    w = words[i]
    if w not in w_wordToIdx:
        continue
    if i > 0 and words[i-1] in c_wordToIdx:
        pr_cw_count[w_wordToIdx[w]][c_wordToIdx[words[i-1]]] += 1
    if i > 1 and words[i-2] in c_wordToIdx:
        pr_cw_count[w_wordToIdx[w]][c_wordToIdx[words[i-2]]] += 1
    if i < wordsLength - 1 and words[i+1] in c_wordToIdx:
        pr_cw_count[w_wordToIdx[w]][c_wordToIdx[words[i+1]]] += 1
    if i < wordsLength - 2 and words[i+2] in c_wordToIdx:
        pr_cw_count[w_wordToIdx[w]][c_wordToIdx[words[i+2]]] += 1    

In [95]:
pr_cw = pr_cw_count/pr_cw_count.sum(axis = 1)[:, None]
pr_cw[np.isnan(pr_cw)] = 0 # in case of no c in any window of w

In [179]:
# for i in xrange(0, numC):
#     pr_c_count[i] = wordCount[c_words[i]]
# # pr_c = pr_c_count/np.sum(pr_c_count)
# pr_c = pr_c_count/len(words)
pr_c = pr_cw_count.sum(axis = 0)[:, None]/pr_cw_count.sum(axis = 0)[:, None].sum()

In [181]:
representation = np.maximum(np.zeros(pr_cw.shape), np.log(pr_cw/pr_c.T))

/Library/Python/2.7/site-packages/ipykernel/__main__.py:1: RuntimeWarning: divide by zero encountered in log
  if __name__ == '__main__':


In [182]:
print np.all(np.isfinite(representation))
print np.any(np.isnan(representation))

True
False


In [183]:
# Down to 100 dimensinal representation
pca = PCA(n_components = 100)
pca.fit(representation)
reducedRepresentation = pca.transform(representation)

In [184]:
reducedRepresentation.shape

(5000, 100)

In [199]:
def findNearestNeighbor(wordVector, vocabularyVector, k, distance = 'cosine'):
    minIdx = -1
    minDistance = sys.maxint
    distanceDict = {}
    for i in range(0, len(vocabularyVector)):
        if distance == 'cosine':
            wordDistance = cosine_distance(wordVector, vocabularyVector[i])
        elif distance == 'euclidean':
            wordDistance = util.euclidean_distance(wordVector, vocabularyVector[i])
        else:
            print "error"
        distanceDict[i] = wordDistance

    orderedDistanceDict = OrderedDict(sorted(distanceDict.items(), key=lambda t: t[1], reverse = False))    
    return orderedDistanceDict.keys()[:k]

In [197]:
testWords = ['communism', 'autumn', 'cigarette', 'pulmonary', 'mankind', 'africa', 'chicago', 'revolution', \
             'september', 'chemical', 'detergent', 'dictionary', 'storm', 'worship', 'face', 'president', \
             'education', 'million', 'face', 'college', 'poet', 'commission', 'sunday', 'children', 'business']
for word in testWords:
    print map(lambda idx: w_words[idx], findNearestNeighbor(reducedRepresentation[w_wordToIdx[word]], reducedRepresentation, 6))

['communism', 'phrase', 'utopian', 'immediate', 'china', 'occupation']
['autumn', 'storm', 'winter', 'summer', 'wines', 'candidates']
['cigarette', 'shut', 'lighted', 'peered', 'nodded', 'seated']
['pulmonary', 'artery', 'bronchial', 'saline', 'lungs', 'distributed']
['mankind', 'struggle', 'life', 'death', 'belief', 'history']
['africa', 'asia', 'western', 'europe', 'america', 'germany']
['chicago', 'club', 'board', 'top', 'boston', 'press']
['revolution', 'perhaps', 'lo', 'guns', 'known', 'hope']
['september', 'june', 'december', 'july', '1960', 'april']
['chemical', 'feed', 'thermal', 'similar', 'results', 'concept']
['detergent', 'fabrics', 'grains', 'saline', 'butter', 'indirect']
['dictionary', 'text', 'occurrence', 'index', 'symbolic', 'stored']
['storm', 'autumn', 'reminded', 'wedding', 'eighteenth', 'clock']
['worship', 'shared', 'conscience', 'beliefs', 'life', 'religion']
['face', 'eyes', 'looked', 'hair', 'turned', 'suddenly']
['president', 'kennedy', 'chairman', 'conferenc

In [200]:
for word in testWords:
    print map(lambda idx: w_words[idx], findNearestNeighbor(reducedRepresentation[w_wordToIdx[word]], reducedRepresentation, 6, distance = 'euclidean'))

['communism', 'utopian', 'phrase', 'world', 'war', 'far']
['autumn', 'storm', 'one', 'first', 'orleans', 'last']
['cigarette', 'shut', 'cathy', 'hurried', 'time', 'still']
['pulmonary', 'artery', 'bronchial', 'reactivity', 'saline', 'lungs']
['mankind', 'life', 'world', 'one', 'must', 'great']
['africa', 'asia', 'europe', 'world', 'america', 'western']
['chicago', 'club', 'place', 'top', 'work', 'brought']
['revolution', 'perhaps', 'known', 'even', 'must', 'much']
['september', 'june', 'first', 'december', 'last', 'november']
['chemical', 'also', 'similar', 'example', 'may', 'given']
['detergent', 'fabrics', 'dishes', 'butter', 'one', 'saline']
['dictionary', 'text', 'occurrence', 'entry', 'symbolic', 'index']
['storm', 'one', 'time', 'eighteenth', 'place', 'clock']
['worship', 'life', 'acts', 'church', 'shared', 'humanity']
['face', 'eyes', 'looked', 'look', 'turned', 'saw']
['president', 'kennedy', 'mr.', 'washington', 'new', 'vice']
['education', 'public', 'national', 'schools', 'pr

In [100]:
# Clustering using kmeans - endu distance
kmeans = KMeans(n_clusters = 100, random_state = 0)
kmeans.fit(reducedRepresentation)

labels = kmeans.labels_
cluster = [[] for _ in xrange(0,100)]
for i in xrange(0,numW):
    cluster[int(labels[i])-1].append(w_words[i])   

In [167]:
gaacCluster =  GAAClusterer(num_clusters = 100)
gaacCluster.cluster(reducedRepresentation)

clusterGaac = [[] for i in xrange(0,100)]
for word in w_words:
    label = int(gaacCluster.classify(reducedRepresentation[w_wordToIdx[word]]))
    cluster[label].append(word)

In [201]:
kmeansCluster = KMeansClusterer(100, cosine_distance)
kmeansCluster.cluster(reducedRepresentation)

In [202]:
clusterKmeans = defaultdict(list)
for word in w_words:
    label = int(kmeansCluster.classify(reducedRepresentation[w_wordToIdx[word]]))
    clusterKmeans[label].append(word)

In [206]:
for i in xrange(0,100):
    print i, clusterKmeans[i]
    print

0 ['opera', 'offers', 'enjoy', 'lots', 'skill', 'afford', 'discover', 'forever', 'intention', 'desirable', 'prepare', 'inevitable', 'healthy', 'presents', 'successfully', 'dancers', 'era', 'dates', 'exciting', 'lesson', 'argued', 'calendar', 'colony', 'absolutely', 'festival', 'suspicion', 'preferred', 'terror', 'wishes', 'habit', 'adults', 'bases', 'motive', 'fortunate', 'associate', 'flash', 'anticipation', 'orderly', 'pioneer', 'attain', 'startling', "people's"]

1 ['new', 'city', 'york', 'town', 'park', 'houses', 'located', 'homes', 'streets', 'plants', 'railroad', 'dallas', 'fort', 'centers', 'los', 'welcome', 'fill', 'avenue', 'georgia', 'pennsylvania', 'manchester', 'tour', 'downtown', 'orleans', 'focus', 'newspapers', 'occupied', 'metropolitan', 'factory', 'parking', 'suburban', 'freight', 'yankees', 'libraries', 'greenwich', 'shopping', 'landing', 'invited', 'tennessee', 'historic', 'orange', 'civic', 'miami', 'bid', 'drinks', 'springs', "city's", 'monument', 'warwick', 'hotel